## Setup

In [7]:
from langchain_groq.chat_models import ChatGroq
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

#Available Models
groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}
model = groq_models["llama3-70b"]

# Constants
green = "\x1b[38;2;152;251;152m"
red = "\x1b[38;2;255;0;0m"
reset = "\x1b[0m"
GROQ_API_KEY = "GROQ_API_KEY"
time = 10
offset = 100
folders = ["LAYING","SITTING","STANDING","WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"]
classes = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}

combined_dir = os.path.join("./HAR/Combined")

X_train=[]
y_train=[]

dataset_dir = os.path.join(combined_dir,"Train")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir,folder))

    for file in files:

        df = pd.read_csv(os.path.join(dataset_dir,folder,file),sep=",",header=0)
        df = df[offset:offset+time*50]
        X_train.append(df.values)
        y_train.append(classes[folder])

X_test=[]
y_test=[]

dataset_dir = os.path.join(combined_dir,"Test")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir,folder))
    for file in files:

        df = pd.read_csv(os.path.join(dataset_dir,folder,file),sep=",",header=0)
        df = df[offset:offset+time*50]
        X_test.append(df.values)
        y_test.append(classes[folder])

# concatenate the training and testing data
X = np.concatenate((X_train,X_test))
y = np.concatenate((y_train,y_test))

# split the data into training and testing sets. Change the seed value to obtain different random splits.
seed = 4
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=seed,stratify=y)

print("Training data shape: ",X_train.shape)
print("Testing data shape: ",X_test.shape)

categories = list(classes.keys())

# train prompt template
train_prompt = lambda data,category: f"""
You do not need to output anything for this query.
Example data for "{category}":
{data}
"""

# test prompt template
test_prompt = lambda data: f"""
You will be provided 3-axis accelerometer data from one of the following activities:
    1) WALKING
    2) LAYING
    3) WALKING_UPSTAIRS
    4) WALKING_DOWNSTAIRS
    5) SITTING
    6) STANDING
You have to ouput only the name of the predicted activity and nothing else.
DATA:
{data}
"""


Training data shape:  (126, 500, 3)
Testing data shape:  (54, 500, 3)


## Zero Shot

In [8]:
llm = ChatGroq(model=model, api_key=GROQ_API_KEY, temperature=0)

correct_count = 0

for i in range(len(X_test)):
    query = test_prompt(X_test[i])
    ans = llm.invoke(query).content
    if(ans==categories[y_test[i]-1]):
        correct_count+=1
        color = green
    else:
        color = red
    print(f"{i}) Actual: {categories[y_test[i]-1]}\n{color}Predicted: {ans}{reset}")

print(f"""
Total: {len(X_test)}
Correct: {correct_count}
Accuracy: {correct_count/len(X_test)}
""")

0) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
1) Actual: WALKING
Predicted: WALKING
2) Actual: WALKING_UPSTAIRS
Predicted: WALKING
3) Actual: STANDING
Predicted: WALKING
4) Actual: STANDING
Predicted: WALKING
5) Actual: WALKING
Predicted: WALKING
6) Actual: WALKING
Predicted: WALKING
7) Actual: STANDING
Predicted: WALKING
8) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
9) Actual: WALKING_UPSTAIRS
Predicted: WALKING
10) Actual: LAYING
Predicted: WALKING
11) Actual: STANDING
Predicted: STANDING
12) Actual: LAYING
Predicted: WALKING
13) Actual: STANDING
Predicted: WALKING
14) Actual: LAYING
Predicted: WALKING
15) Actual: WALKING
Predicted: WALKING
16) Actual: LAYING
Predicted: WALKING
17) Actual: STANDING
Predicted: WALKING
18) Actual: WALKING_UPSTAIRS
Predicted: WALKING
19) Actual: STANDING
Predicted: WALKING
20) Actual: SITTING
Predicted: WALKING
21) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
22) Actual: WALKING_UPSTAIRS
Predicted: WALKING
23) Actual: WALKING_UPSTAIRS
Predicted: 

## Few Shots (Training)

In [9]:
llm = ChatGroq(model=model, api_key=GROQ_API_KEY, temperature=0)

print(llm.invoke("""
You will be provided examples for 3-axis accelerometer data from one of the following activities:
    1) WALKING
    2) LAYING
    3) WALKING_UPSTAIRS
    4) WALKING_DOWNSTAIRS
    5) SITTING
    6) STANDING
You have to learn from these examples and then you will be tested against test dataset.
You do not need to output anything for this query.
""").content)

for i in range(len(X_train)):
    query = train_prompt(X_train[i],categories[y_train[i]-1])
    response = llm.invoke(query)
    #print(response.content)

I'm ready to learn from the examples. Please provide the 3-axis accelerometer data for each of the activities (WALKING, LAYING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING). I'll learn from these examples and prepare to be tested against the test dataset.


## Few Shots (Testing)

In [10]:
correct_count = 0

for i in range(len(X_test)):
    query = test_prompt(X_test[i])
    ans = llm.invoke(query).content
    if(ans==categories[y_test[i]-1]):
        correct_count+=1
        color = green
    else:
        color = red
    print(f"{i}) Actual: {categories[y_test[i]-1]}\n{color}Predicted: {ans}{reset}")

print(f"""Total: {len(X_test)}
Correct: {correct_count}
Accuracy: {correct_count/len(X_test)}
""")

0) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
1) Actual: WALKING
Predicted: WALKING
2) Actual: WALKING_UPSTAIRS
Predicted: WALKING
3) Actual: STANDING
Predicted: WALKING
4) Actual: STANDING
Predicted: WALKING
5) Actual: WALKING
Predicted: WALKING
6) Actual: WALKING
Predicted: WALKING
7) Actual: STANDING
Predicted: WALKING
8) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
9) Actual: WALKING_UPSTAIRS
Predicted: WALKING
10) Actual: LAYING
Predicted: WALKING
11) Actual: STANDING
Predicted: STANDING
12) Actual: LAYING
Predicted: WALKING
13) Actual: STANDING
Predicted: WALKING
14) Actual: LAYING
Predicted: WALKING
15) Actual: WALKING
Predicted: WALKING
16) Actual: LAYING
Predicted: WALKING
17) Actual: STANDING
Predicted: WALKING
18) Actual: WALKING_UPSTAIRS
Predicted: WALKING
19) Actual: STANDING
Predicted: WALKING
20) Actual: SITTING
Predicted: WALKING
21) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
22) Actual: WALKING_UPSTAIRS
Predicted: WALKING
23) Actual: WALKING_UPSTAIRS
Predicted: 

## Random Test

In [13]:
x_min = np.min(X_test)
x_max = np.max(X_test)
y_min = np.min(y_test)
y_max = np.max(y_test)

X_test = (x_max-x_min)*np.random.random(len(X_test)) + x_min
y_test = np.random.randint(y_min,y_max+1,len(X_test))

correct_count = 0

for i in range(len(X_test)):
    query = test_prompt(X_test[i])
    ans = llm.invoke(query).content
    if(ans==categories[y_test[i]-1]):
        correct_count+=1
        color = green
    else:
        color = red
    print(f"{i}) Actual: {categories[y_test[i]-1]}\n{color}Predicted: {ans}{reset}")

print(f"""Total: {len(X_test)}
Correct: {correct_count}
Accuracy: {correct_count/len(X_test)}
""")

0) Actual: LAYING
Predicted: STANDING
1) Actual: WALKING_UPSTAIRS
Predicted: STANDING
2) Actual: STANDING
Predicted: STANDING
3) Actual: WALKING_UPSTAIRS
Predicted: STANDING
4) Actual: STANDING
Predicted: STANDING
5) Actual: WALKING_UPSTAIRS
Predicted: STANDING
6) Actual: SITTING
Predicted: STANDING
7) Actual: STANDING
Predicted: STANDING
8) Actual: WALKING_DOWNSTAIRS
Predicted: STANDING
9) Actual: WALKING
Predicted: STANDING
10) Actual: LAYING
Predicted: STANDING
11) Actual: LAYING
Predicted: STANDING
12) Actual: WALKING
Predicted: STANDING
13) Actual: SITTING
Predicted: STANDING
14) Actual: SITTING
Predicted: STANDING
15) Actual: LAYING
Predicted: STANDING
16) Actual: LAYING
Predicted: STANDING
17) Actual: STANDING
Predicted: STANDING
18) Actual: WALKING_UPSTAIRS
Predicted: STANDING
19) Actual: WALKING_UPSTAIRS
Predicted: STANDING
20) Actual: STANDING
Predicted: STANDING
21) Actual: WALKING_DOWNSTAIRS
Predicted: STANDING
22) Actual: SITTING
Predicted: STANDING
23) Actual: WALKING_DOW